# Function Calling Tutorial PT 3

Okay in this example I'm going to ask for the weather in both Sydney and Brazil as well as the time so I'm going to add a new function as well as some extra code to get result from new function.

In [24]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
import requests

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [25]:
!pip install pytz

In [26]:
import pytz
from datetime import datetime

def get_current_time(timezone):
    """
    Fetches the current time in the specified timezone.
    """
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S %Z")
        return current_time
    except pytz.UnknownTimeZoneError:
        return "Invalid timezone. Please provide a valid IANA timezone."

print(get_current_time("Australia/Sydney"))

2025-09-16 01:50:41 AEST


In [27]:
# Define the function you are going to use:
# Uses Open-Meteo's Weather Forecast API!!!
def get_weather(latitude, longitude):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m"
    )
    
    response = requests.get(url)
    data = response.json()
    
    return data.get('current').get('temperature_2m')

# example use
get_weather("-33.8727", "151.2057")

14.6

In [28]:
# define list of callable tools for model
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in degrees Celcius for a given latitude and longitude",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "The latitude of a place",
                },
                "longitude": {
                    "type": "string",
                    "description": "The longitude of a place",
                },
            },
            "required": ["latitude", "longitude"],
        }
    },
    {
        "type": "function",
        "name": "get_current_time",
        "description": "Get the current time for given IANA timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "IANA timezone": {
                    "type": "string",
                    "description": "The IANA timezone of a place",
                },
            },
            "required": ["IANA timezone"]
        }
    }
]

messages = [{"role": "user", "content": "What is the weather and current time in Sydney and Busan today?"}]

response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
    tool_choice="auto",
)

#print(response.model_dump_json(indent=2))
print(response.output)

# WHAT HAPPENED?
# We called the api endpoint and it returned which tool we should use and the inputs

[ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_B5HzbkhAYEWcjWI8JUjuhl4O', name='get_current_weather', type='function_call', id='fc_0a0074ae0e3c070c0068c835d5d23881a29cb8f5428a11d273', status='completed'), ResponseFunctionToolCall(arguments='{"IANA timezone":"Australia/Sydney"}', call_id='call_9pR7v882yCj9smQ2Iwrypg2T', name='get_current_time', type='function_call', id='fc_0a0074ae0e3c070c0068c835d6112881a2b7b4ac1281d47b09', status='completed'), ResponseFunctionToolCall(arguments='{"latitude":"35.1796","longitude":"129.0756"}', call_id='call_U9UcQqtmtbeep1GMlgBlhT14', name='get_current_weather', type='function_call', id='fc_0a0074ae0e3c070c0068c835d6425c81a28348a15f1c6fe236', status='completed'), ResponseFunctionToolCall(arguments='{"IANA timezone":"Asia/Seoul"}', call_id='call_72YPdArlGbYtXaiOEBkEwQ7k', name='get_current_time', type='function_call', id='fc_0a0074ae0e3c070c0068c835d6849c81a2b8814ea74f733f8d', status='completed')]


In [29]:
function_call = None
function_call_arguments = None
messages += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        function_call_arguments = json.loads(item.arguments)

        print(function_call)
        print(function_call_arguments)

        # 3. Execute the function logic for each function:
        if function_call.name == "get_current_weather":
            result = {"weather": get_weather(function_call_arguments["latitude"], 
                                             function_call_arguments["longitude"])}

        if function_call.name == "get_current_time":
            result = {"time": get_current_time(function_call_arguments["IANA timezone"])}
        
        print(result)

        # 4. Provide function call results to the model
        messages.append({
            "type": "function_call_output",
            "call_id": function_call.call_id,
            "output": json.dumps(result),
        })

        from pprint import pprint
        pprint(messages)


ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_B5HzbkhAYEWcjWI8JUjuhl4O', name='get_current_weather', type='function_call', id='fc_0a0074ae0e3c070c0068c835d5d23881a29cb8f5428a11d273', status='completed')
{'latitude': '-33.8688', 'longitude': '151.2093'}
{'weather': 14.3}
[{'content': 'What is the weather and current time in Sydney and Busan today?',
  'role': 'user'},
 ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_B5HzbkhAYEWcjWI8JUjuhl4O', name='get_current_weather', type='function_call', id='fc_0a0074ae0e3c070c0068c835d5d23881a29cb8f5428a11d273', status='completed'),
 ResponseFunctionToolCall(arguments='{"IANA timezone":"Australia/Sydney"}', call_id='call_9pR7v882yCj9smQ2Iwrypg2T', name='get_current_time', type='function_call', id='fc_0a0074ae0e3c070c0068c835d6112881a2b7b4ac1281d47b09', status='completed'),
 ResponseFunctionToolCall(arguments='{"latitude":"35.1796","longitude":"12

In [30]:
response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.output_text + "\n\n")
print(response.model_dump_json(indent=2))


Final output:
The current weather in Sydney is 14.3°C, and the current time is 01:50 AM on September 16th, 2025 (AEST).

The current weather in Busan is 23.0°C, and the current time is 12:50 AM on September 16th, 2025 (KST).


{
  "id": "resp_0a0074ae0e3c070c0068c835da6de881a2bbdf4dd44e8f5e84",
  "created_at": 1757951450.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "id": "msg_0a0074ae0e3c070c0068c835ded1c481a2982a9837e51d451d",
      "content": [
        {
          "annotations": [],
          "text": "The current weather in Sydney is 14.3°C, and the current time is 01:50 AM on September 16th, 2025 (AEST).\n\nThe current weather in Busan is 23.0°C, and the current time is 12:50 AM on September 16th, 2025 (KST).",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type":